In [1]:
import pandas as pd
import numpy as np
import os
import math
import random
import re

# for plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# for datetime operations
from dateutil.parser import parse
from datetime import timedelta
from datetime import datetime

from GlucoCheck.glucoCheck import glucoCheckOps
glucoCheckObject = glucoCheckOps()

Using TensorFlow backend.


In [2]:
data = glucoCheckObject.hall_data
data['Display Time'] = pd.to_datetime(data['Display Time']) # Ensures that displayTime column 
# is converted to datetime format.
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])# Ensures that gluvoseValue is numeric
data.head() # this allows you to peek at loaded data; or type just data to see first and last portion of your dataset

,subjectId,Display Time,GlucoseValue
0,1636-69-001,2014-02-03 03:42:00,93
1,1636-69-001,2014-02-03 03:47:00,93
2,1636-69-001,2014-02-03 03:52:00,93
3,1636-69-001,2014-02-03 03:57:00,95
4,1636-69-001,2014-02-03 04:02:00,96


In [3]:
uid = '1636-69-028' 
new = data[data.iloc[:,0]==str(uid)]
new = new.reset_index(drop=True)
# new['GlucoseValue'] = new.iloc[:,2]
# new = new.astype({'GlucoseValue':int})
# new['GV'] = new.iloc[:,2]
# new['DT'] = pd.to_datetime(new.iloc[:,1])
# new['DT'] = new['DT'].dt.round("5min")
# display(new)

In [4]:
new.head()

,subjectId,Display Time,GlucoseValue
0,1636-69-028,2015-04-02 13:33:00,129
1,1636-69-028,2015-04-02 13:38:00,131
2,1636-69-028,2015-04-02 13:43:00,132
3,1636-69-028,2015-04-02 13:48:00,137
4,1636-69-028,2015-04-02 13:58:00,133


In [20]:
new['EWM-GV'] = new.GlucoseValue.ewm(alpha=0.93, adjust=False).mean()
new['EWM-GV'] = new['EWM-GV'].round()
new

,subjectId,Display Time,GlucoseValue,EWM-GV
0,1636-69-028,2015-04-02 13:33:00,129,129.0
1,1636-69-028,2015-04-02 13:38:00,131,131.0
2,1636-69-028,2015-04-02 13:43:00,132,132.0
3,1636-69-028,2015-04-02 13:48:00,137,137.0
4,1636-69-028,2015-04-02 13:58:00,133,133.0
...,...,...,...,...
1817,1636-69-028,2015-06-16 20:58:00,93,93.0
1818,1636-69-028,2015-06-16 21:03:00,94,94.0
1819,1636-69-028,2015-06-16 21:08:00,90,90.0
1820,1636-69-028,2015-06-16 21:13:00,87,87.0


In [21]:
new['EWMA-GV'] = new.GlucoseValue.ewma(alpha=0.93, adjust=False).mean()
# new['EWM-GV'] = new['EWM-GV'].round()
new

AttributeError: 'Series' object has no attribute 'ewma'

In [ ]:
start_x = new.DT[0]
end_x = new.DT.iat[-1]
start_x, end_x

In [ ]:
x_vals = pd.date_range(start=start_x,end=end_x,freq='5T')


In [ ]:
counter = 0
g = new['GV'].tolist()
l = new['DT'].tolist()
y_vals = []
for x in x_vals:
    if x in l:
        y_vals.append(g[counter])
        counter+=1
    else:
        y_vals.append(float("NaN"))


In [ ]:
df = {'X':x_vals, 'Y':y_vals}
df = pd.DataFrame(df)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

plot = sns.lineplot(
    ax=ax,
    data=df, 
    x="X", y="Y",
    hue=df["Y"].isna().cumsum(), 
    palette=["cornflowerblue"]*df["Y"].isna().cumsum().nunique(),
    legend=False, markers=False
)
plot.set_xlabel('Timestamp', weight='bold', fontsize=14)
plot.set_ylabel('Glucose Value', weight='bold', fontsize=14)
# ax.set_xticklabels([])

plt.show()

In [ ]:
df

In [ ]:
# creating gaps
def createGap(df,start,end):
    l = len(df.index)
    if end>=l:
        end = l
        
    for i in range(start,end):
        df['GlucoseValue'][i]= float("NaN")
    
    return df


seed = 1250
start = seed
end = seed+4

dataWithMissing = data.copy()
dataWithMissing = createGap(dataWithMissing,start,end)

In [ ]:
dataWithMissing.GlucoseValue[1250]

In [ ]:
dataWithMissing.to_csv("~/Desktop/NCSA_genomics/testfile_LINEAR.csv")

In [1]:
import pandas as pd
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Data/Hall/data_hall_raw.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])
data = data[data['subjectId']=='1636-69-001']

In [2]:
data

,subjectId,Display Time,GlucoseValue
0,1636-69-001,2014-02-03 03:42:00,93
1,1636-69-001,2014-02-03 03:47:00,93
2,1636-69-001,2014-02-03 03:52:00,93
3,1636-69-001,2014-02-03 03:57:00,95
4,1636-69-001,2014-02-03 04:02:00,96
...,...,...,...
1841,1636-69-001,2015-04-02 14:48:00,115
1842,1636-69-001,2015-04-02 14:53:00,114
1843,1636-69-001,2015-04-02 14:58:00,117
1844,1636-69-001,2015-04-02 15:03:00,121


In [3]:
from GlucoCheck.glucoCheck import glucoCheckOps # to load glucoCheck package

glucoCheckObject = glucoCheckOps()

Using TensorFlow backend.


In [4]:
x,y = glucoCheckObject.splitIndividual(data,30)

In [5]:
x

,subjectId,Display Time,GlucoseValue,time_gap,gaps
0,1636-69-001-1,2014-02-03 03:42:00,93,00:00:00,NaN
1,1636-69-001-1,2014-02-03 03:47:00,93,00:05:00,1.0
2,1636-69-001-1,2014-02-03 03:52:00,93,00:05:00,1.0
3,1636-69-001-1,2014-02-03 03:57:00,95,00:05:00,1.0
4,1636-69-001-1,2014-02-03 04:02:00,96,00:05:00,1.0
...,...,...,...,...,...
749,1636-69-001-1,2014-02-05 18:11:00,98,00:05:00,1.0
750,1636-69-001-1,2014-02-05 18:16:00,99,00:05:00,1.0
751,1636-69-001-1,2014-02-05 18:21:00,99,00:05:00,1.0
752,1636-69-001-1,2014-02-05 18:26:00,100,00:05:00,1.0


In [6]:
y

,subjectId,Display Time,GlucoseValue,time_gap,gaps
754,1636-69-001-1-2,2015-03-29 11:53:00,100,416 days 17:22:00,120016.4
755,1636-69-001-1-2,2015-03-29 11:58:00,111,0 days 00:05:00,1.0
756,1636-69-001-1-2,2015-03-29 12:03:00,113,0 days 00:05:00,1.0
757,1636-69-001-1-2,2015-03-29 12:08:00,114,0 days 00:05:00,1.0
758,1636-69-001-1-2,2015-03-29 12:13:00,116,0 days 00:05:00,1.0
...,...,...,...,...,...
1841,1636-69-001-1-2,2015-04-02 14:48:00,115,0 days 00:05:00,1.0
1842,1636-69-001-1-2,2015-04-02 14:53:00,114,0 days 00:05:00,1.0
1843,1636-69-001-1-2,2015-04-02 14:58:00,117,0 days 00:05:00,1.0
1844,1636-69-001-1-2,2015-04-02 15:03:00,121,0 days 00:05:00,1.0
